# Manage Quota

Note: 
- Geo Engine has to have "pro" feature enabled
- quota_check has to be enabled in Settings

In [1]:
import geoengine as ge

from datetime import datetime

# Select a user

In [2]:
email = "foo@example.com"
password = "secret123"

# register the user, if it doesn't exist yet
import requests as re

re.post("http://localhost:3030/api/user", json={"email": email, "password": password, "realName": "Foo Bar"})

<Response [200]>

## Initialize as user

In [3]:
ge.initialize("http://localhost:3030/api", credentials=(email, password))

user_id = ge.get_session().user_id

## Access own quota

In [4]:
ge.get_quota()

Quota(available=0, used=0)

## Try to run a query (fails, because quota is exhausted)

In [5]:
ports = ge.register_workflow(
    {
        "type": "Vector",
        "operator": {"type": "OgrSource", "params": {"data": "ne_10m_ports", "attributeProjection": None}},
    }
)

time = datetime.strptime("2014-04-01T12:00:00.000Z", "%Y-%m-%dT%H:%M:%S.%f%z")

try:
    df = ports.get_dataframe(
        ge.QueryRectangle(
            ge.BoundingBox2D(-180.0, -90.0, 180.0, 90.0), ge.TimeInterval(time, time), ge.SpatialResolution(0.1, 0.1)
        )
    )
except ge.GeoEngineException as e:
    print(e)

## Initialize Geo Engine as Admin

In [6]:
ge.initialize("http://localhost:3030/api", ("admin@localhost", "adminadmin"))

## Access user quota

In [7]:
ge.get_quota(user_id)

Quota(available=0, used=0)

## Update user quota

In [8]:
ge.update_quota(user_id, 1000)

## Verify quota update worked

In [9]:
ge.get_quota(user_id)

Quota(available=1000, used=0)

# Go back to the regular user

In [10]:
ge.initialize("http://localhost:3030/api", credentials=(email, password))

ge.get_quota()

Quota(available=1000, used=0)

# Rerun the workflow, works now

In [11]:
df = ports.get_dataframe(
    ge.QueryRectangle(
        ge.BoundingBox2D(-180.0, -90.0, 180.0, 90.0), ge.TimeInterval(time, time), ge.SpatialResolution(0.1, 0.1)
    )
)

df.head()

,geometry,natlscale,website,scalerank,featurecla,name,start,end
0,POINT (-69.92356 12.4375),5.0,www.rocargo.com/SanNicolas.html,8,Port,Sint Nicolaas,NaT,NaT
1,POINT (-58.95141 -34.15333),5.0,www.consejoportuario.com.ar,8,Port,Campana,NaT,NaT
2,POINT (-59.00495 -34.09889),5.0,www.consejoportuario.com.ar,8,Port,Zarate,NaT,NaT
3,POINT (-62.10088 -38.89444),5.0,None,8,Port,Puerto Belgrano/Bahia Blanca,NaT,NaT
4,POINT (-62.30053 -38.78306),5.0,None,8,Port,Puerto Galvan/Bahia Blanca,NaT,NaT


## Verify that the used quota was recorded

In [12]:
ge.get_quota()

Quota(available=1000, used=0)